# AutoDL - Regression

## Imports and Global Settings

In [1]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import tensorflow as tf
import autokeras as ak
from sklearn.metrics import mean_absolute_error, r2_score

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import (
    ModelSetup,
    evaluate_reg_model,
    calculate_roi,
    save_model_report,
)

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

2023-10-30 06:39:46.925102: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 06:39:46.988270: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 06:39:46.989196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 06:39:48.105777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load Data

In [2]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [3]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [4]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [5]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df = games_features.copy()

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3911 entries, 0 to 3910
Data columns (total 763 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3911 non-null   object        
 1    game_datetime                                 3911 non-null   datetime64[ns]
 2    home_team                                     3911 non-null   object        
 3    away_team                                     3911 non-null   object        
 4    open_line                                     3902 non-null   float64       
 5    home_score                                    3895 non-null   float64       
 6    away_score                                    3895 non-null   float64       
 7    game_completed                                3911 non-null   bool          
 8    odds_last_update                              41 non-nul

In [8]:
df.head(10)

game_id       game_datetime home_team away_team  open_line  \
0  20230529BOSMIA 2023-05-29 20:30:00       BOS       MIA       -7.5   
1  20230427ATLBOS 2023-04-27 20:30:00       ATL       BOS        9.5   
2  20230428GSWSAC 2023-04-28 20:00:00       GSW       SAC       -7.0   
3  20230428LALMEM 2023-04-28 22:30:00       LAL       MEM       -5.0   
4  20230504GSWLAL 2023-05-04 21:00:00       GSW       LAL       -7.5   
5  20230502NYKMIA 2023-05-02 19:30:00       NYK       MIA      -10.0   
6  20230502GSWLAL 2023-05-02 22:00:00       GSW       LAL       -5.5   
7  20230429DENPHX 2023-04-29 20:30:00       DEN       PHX       -3.5   
8  20230501BOSPHI 2023-05-01 19:30:00       BOS       PHI      -10.5   
9  20230501DENPHX 2023-05-01 22:00:00       DEN       PHX       -4.0   

   home_score  away_score  game_completed odds_last_update scores_last_update  \
0        84.0       103.0            True              NaT                NaT   
1       120.0       128.0            True              NaT                NaT   
2        99.0       118.0            True              NaT                NaT   
3       125.0        85.0            True              NaT                NaT   
4       127.0       100.0            True              NaT                NaT   
5       111.0       105.0            True              NaT                NaT   
6       112.0       117.0            True              NaT                NaT   
7       125.0       107.0            True              NaT                NaT   
8       115.0       119.0            True              NaT                NaT   
9        97.0        87.0            True              NaT                NaT   

      season  away_ATL  away_BKN  away_BOS  away_CHA  away_CHI  away_CLE  \
0  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
1  2022-2023       0.0       0.0       1.0       0.0       0.0       0.0   
2  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
3  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
4  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
5  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
6  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
7  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
8  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
9  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   

   away_DAL  away_DEN  away_DET  away_GSW  away_HOU  away_IND  away_LAC  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
6       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
7       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
8       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
9       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   away_LAL  away_MEM  away_MIA  away_MIL  away_MIN  away_NOP  away_NYK  \
0       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       1.0       0.0       0.0       0.0       0.0       0.0   
4       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
5       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
6       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
7       0.0       0.0       0.0       0.0       0.0       0.0 

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [9]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

### Create Targets

In [10]:
df = ModelSetup.add_targets(df)

### Select Features

In [11]:
training_seasons = [x for x in range(2020, 2022)]
training_dates, testing_dates = ModelSetup.choose_dates(training_seasons, [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2020-12-22', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [12]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
away_ATL
away_BKN
away_BOS
away_CHA
away_CHI
away_CLE
away_DAL
away_DEN
away_DET
away_GSW
away_HOU
away_IND
away_LAC
away_LAL
away_MEM
away_MIA
away_MIL
away_MIN
away_NOP
away_NYK
away_OKC
away_ORL
away_PHI
away_PHX
away_POR
away_SAC
away_SAS
away_TOR
away_UTA
away_WAS
home_ATL
home_BKN
home_BOS
home_CHA
home_CHI
home_CLE
home_DAL
home_DEN
home_DET
home_GSW
home_HOU
home_IND
home_LAC
home_LAL
home_MEM
home_MIA
home_MIL
home_MIN
home_NOP
home_NYK
home_OKC
home_ORL
home_PHI
home_PHX
home_POR
home_SAC
home_SAS
home_TOR
home_UTA
home_WAS
last_5_hv
streak_hv
win_pct_hv
season_type
rest_diff_hv
day_of_season
point_diff_hv
away_team_streak
home_team_streak
away_team_win_pct
home_team_win_pct
point_diff_last_5_hv
pie_away_all_advanced
pie_away_l2w_advanced
pie_home_all_advanced
pie_home_l2w_advanced
l_away_all_traditional
l_away_l2w_traditional
l_home_all_traditio

In [13]:
features_to_use = [
    "open_line",
    "rest_diff_hv",
    "day_of_season",
    "last_5_hv",
    "streak_hv",
    "point_diff_last_5_hv",
    "point_diff_hv",
    "win_pct_hv",
    "pie_percentile_away_all_advanced",
    "home_team_avg_point_diff",
    "net_rating_away_all_advanced",
    "net_rating_home_all_advanced",
    "plus_minus_home_all_traditional",
    "e_net_rating_zscore_away_all_advanced",
    "net_rating_zscore_away_all_advanced",
    "plus_minus_away_all_opponent",
    "away_team_avg_point_diff",
    "plus_minus_away_all_traditional",
    "pie_zscore_away_all_advanced",
    "e_net_rating_away_all_advanced",
    "plus_minus_percentile_away_all_traditional",
    "net_rating_zscore_home_l2w_advanced",
    "e_net_rating_home_all_advanced",
    "w_zscore_away_all_traditional",
    "pie_away_all_advanced",
    "w_pct_zscore_away_all_traditional",
    "e_net_rating_percentile_away_l2w_advanced",
]

In [14]:
df.dropna(subset=features_to_use, inplace=True)

In [15]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "reg", features_to_use, training_dates, testing_dates, create_report=True
)

In [16]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2353, 30)
Testing Shape:  (1214, 30)


### Baselines

In [17]:
training_baseline_via_vegas = model_report["ind_baseline_train"]
testing_baseline_via_vegas = model_report["ind_baseline_test"]

training_baseline_via_mean = model_report["dep_baseline_train"]
testing_baseline_via_mean = model_report["dep_baseline_test"]

print(f"Training Baseline via Vegas: {training_baseline_via_vegas:.2f}")
print(f"Testing Baseline via Vegas: {testing_baseline_via_vegas:.2f}")
print(f"Training Baseline via Mean: {training_baseline_via_mean:.2f}")
print(f"Testing Baseline via Mean: {testing_baseline_via_mean:.2f}")

Training Baseline via Vegas: 10.69
Testing Baseline via Vegas: 9.78
Training Baseline via Mean: 12.20
Testing Baseline via Mean: 11.00


<a id=Regression></a>

## Regression

In [18]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2353 entries, 152 to 2550
Data columns (total 30 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   game_id                                     2353 non-null   object 
 1   vegas_open_hv                               2353 non-null   float64
 2   REG_TARGET                                  2353 non-null   float64
 3   open_line                                   2353 non-null   float64
 4   rest_diff_hv                                2353 non-null   float64
 5   day_of_season                               2353 non-null   float64
 6   last_5_hv                                   2353 non-null   float64
 7   streak_hv                                   2353 non-null   float64
 8   point_diff_last_5_hv                        2353 non-null   float64
 9   point_diff_hv                               2353 non-null   float64
 10  win_pct_hv

In [19]:
X_train = training_df[features_to_use]
X_test = testing_df[features_to_use]
y_train = training_df["REG_TARGET"]
y_test = testing_df["REG_TARGET"]

## AutoKeras

In [20]:
reg = ak.StructuredDataRegressor(
    max_trials=10,
    overwrite=True,
    loss="mean_absolute_error",
)

In [21]:
reg.fit(X_train, y_train)

Trial 10 Complete [00h 00m 10s]
val_loss: 11.476484298706055

Best val_loss So Far: 11.369766235351562
Total elapsed time: 00h 02m 37s
Epoch 1/14
74/74 [==============================] - 1s 3ms/step - loss: 11.9546 - mean_squared_error: 222.4529
Epoch 2/14
74/74 [==============================] - 0s 3ms/step - loss: 11.4216 - mean_squared_error: 207.3789
Epoch 3/14
74/74 [==============================] - 0s 3ms/step - loss: 11.2520 - mean_squared_error: 203.3120
Epoch 4/14
74/74 [==============================] - 0s 3ms/step - loss: 11.1787 - mean_squared_error: 201.6683
Epoch 5/14
74/74 [==============================] - 0s 3ms/step - loss: 11.1342 - mean_squared_error: 200.2742
Epoch 6/14
74/74 [==============================] - 0s 3ms/step - loss: 11.0962 - mean_squared_error: 198.9857
Epoch 7/14
74/74 [==============================] - 0s 3ms/step - loss: 11.0612 - mean_squared_error: 197.9571
Epoch 8/14
74/74 [==============================] - 0s 3ms/step - loss: 11.0301 - mean_s

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [22]:
# Evaluate the best model with testing data.
print(reg.evaluate(X_test, y_test))

38/38 [==============================] - 1s 3ms/step - loss: 10.2415 - mean_squared_error: 168.9917
[10.2415189743042, 168.99169921875]


In [23]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 27)]              0         
                                                                 
 multi_category_encoding (M  (None, 27)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 27)                55        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                896       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [24]:
model_report["details"] = model.get_config()

<a id=evaluate></a>

### Evaluate Model

In [25]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

38/38 [==============================] - 0s 4ms/step


In [26]:
train_mae = mean_absolute_error(y_train, train_pred)
train_r2 = r2_score(y_train, train_pred)
print(f"Training MAE: {train_mae:.2f}")
print(f"Training R2: {train_r2:.2f}")

Training MAE: 10.84
Training R2: 0.17


In [27]:
model_report["train_mae"] = train_mae
model_report["train_r2"] = train_r2

In [28]:
test_mae = mean_absolute_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)
print(f"Testing MAE: {test_mae:.2f}")
print(f"Testing R2: {test_r2:.2f}")

Testing MAE: 10.24
Testing R2: 0.10


In [29]:
model_report["test_mae"] = test_mae
model_report["test_r2"] = test_r2

In [30]:
vegas_pred_train = -X_train["open_line"]
train_pred = train_pred.flatten()
train_pred_results = pd.DataFrame(
    {"vegas_open_hv": vegas_pred_train, "model_pred": train_pred, "actual": y_train}
)

In [31]:
vegas_pred_test = -X_test["open_line"]
test_pred = test_pred.flatten()
test_pred_results = pd.DataFrame(
    {"vegas_open_hv": vegas_pred_test, "model_pred": test_pred, "actual": y_test}
)

In [32]:
train_acc_reg, train_closer_to_target_reg, train_prediction_df_reg = evaluate_reg_model(
    train_pred_results, "vegas_open_hv", "actual", "model_pred"
)
test_acc_reg, test_closer_to_target_reg, test_prediction_df_reg = evaluate_reg_model(
    test_pred_results, "vegas_open_hv", "actual", "model_pred"
)

Prediction is closer to target in 49.00% of cases
Accuracy: 0.5295
Prediction is closer to target in 45.55% of cases
Accuracy: 0.5099


In [33]:
model_report["train_acc_reg"] = train_acc_reg
model_report["test_acc_reg"] = test_acc_reg
model_report["train_ctt"] = train_closer_to_target_reg
model_report["test_ctt"] = test_closer_to_target_reg

In [34]:
roi_results_reg = calculate_roi(test_prediction_df_reg, "actual_side", "pred_side")
roi_results_reg

,Label,Total ROI,Average ROI per Bet
0,"All Bets, Even Amount",2400,1.98
1,"All Bets, Typical Odds",-3171,-2.61


In [35]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]
model_report["roi_all_bets_typical_odds_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

### Model Storage

In [36]:
platform = "autokeras"
problem_type = "reg"
model_type = "dl"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")
model_report["datetime"] = model_report["datetime"].strftime("%Y-%m-%d %H:%M:%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

'autokeras_reg_dl_2023_10_30_06_40_04'

In [37]:
model.save(f"../models/AutoDL/{model_id}", save_format="tf")

INFO:tensorflow:Assets written to: ../models/AutoDL/autokeras_reg_dl_2023_10_30_06_40_04/assets


INFO:tensorflow:Assets written to: ../models/AutoDL/autokeras_reg_dl_2023_10_30_06_40_04/assets


In [38]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [39]:
model_report

{'datetime': '2023-10-30 06:40:04',
 'problem_type': 'reg',
 'target': 'REG_TARGET',
 'features': ['open_line',
  'rest_diff_hv',
  'day_of_season',
  'last_5_hv',
  'streak_hv',
  'point_diff_last_5_hv',
  'point_diff_hv',
  'win_pct_hv',
  'pie_percentile_away_all_advanced',
  'home_team_avg_point_diff',
  'net_rating_away_all_advanced',
  'net_rating_home_all_advanced',
  'plus_minus_home_all_traditional',
  'e_net_rating_zscore_away_all_advanced',
  'net_rating_zscore_away_all_advanced',
  'plus_minus_away_all_opponent',
  'away_team_avg_point_diff',
  'plus_minus_away_all_traditional',
  'pie_zscore_away_all_advanced',
  'e_net_rating_away_all_advanced',
  'plus_minus_percentile_away_all_traditional',
  'net_rating_zscore_home_l2w_advanced',
  'e_net_rating_home_all_advanced',
  'w_zscore_away_all_traditional',
  'pie_away_all_advanced',
  'w_pct_zscore_away_all_traditional',
  'e_net_rating_percentile_away_l2w_advanced'],
 'training_start_date': '2020-12-22',
 'training_end_date'

In [40]:
save_model_report(model_report)